In [1]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain,s

In [2]:
from datasets import load_dataset, DatasetDict

In [3]:
import pandas as pd

# Load the dataset
data = pd.read_csv("natural_language_augmented.csv", sep=";")

# Combine the English columns into a single list
# text = [sentence for trio in zip(data["English 1"], data["English 2"], data["English 3"]) for sentence in trio]
text = data["Latex"]
text = [item for item in text[:556] for _ in range(3)]

audio_dirs = [f"audios/v{i}_f{j}.mp3" for j in range(1, 557) for i in range(1, 4)]

In [4]:
from datasets import Dataset, DatasetDict, Audio

# Assuming english_sentences and english_ids are already defined
data_dict = {"sentence": text, "audio": audio_dirs}
dataset = Dataset.from_dict(data_dict)
dataset = dataset.cast_column("audio", Audio())


In [5]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small"
    # "openai/whisper-small", language="english", task="transcribe"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
# common_voice = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [7]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [8]:
import librosa

def downsample_audio(example):
    audio_array, _ = librosa.load(example["audio"]["path"], sr=16000)
    example["audio"]["array"] = audio_array
    example["audio"]["sampling_rate"] = 16000
    return example

dataset = dataset.map(downsample_audio, num_proc=1)

Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

In [9]:
dataset = dataset.map(
    prepare_dataset, num_proc=1
)

Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

In [10]:
dataset = dataset.train_test_split(test_size=0.3)

In [11]:
dataset.save_to_disk("dataset")

Saving the dataset (0/3 shards):   0%|          | 0/1167 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/501 [00:00<?, ? examples/s]

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [14]:
import evaluate

metric = evaluate.load("wer")

In [16]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [17]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [18]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="sinhalese", task="transcribe", use_cache=True
)

In [21]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-latex",  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

/Users/lichu/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]

/Users/lichu/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 5.5848, 'grad_norm': 30.002817153930664, 'learning_rate': 5e-06, 'epoch': 0.34}
{'loss': 2.1397, 'grad_norm': 15.976034164428711, 'learning_rate': 1e-05, 'epoch': 0.68}
{'loss': 1.1272, 'grad_norm': 9.795811653137207, 'learning_rate': 1e-05, 'epoch': 1.03}
{'loss': 0.8242, 'grad_norm': 9.336149215698242, 'learning_rate': 1e-05, 'epoch': 1.37}
{'loss': 0.546, 'grad_norm': 11.363210678100586, 'learning_rate': 1e-05, 'epoch': 1.71}
{'loss': 0.1498, 'grad_norm': 6.645379543304443, 'learning_rate': 1e-05, 'epoch': 2.05}
{'loss': 0.1013, 'grad_norm': 3.200684070587158, 'learning_rate': 1e-05, 'epoch': 2.4}
{'loss': 0.1, 'grad_norm': 5.467233657836914, 'learning_rate': 1e-05, 'epoch': 2.74}
{'loss': 0.081, 'grad_norm': 5.238300800323486, 'learning_rate': 1e-05, 'epoch': 3.08}
{'loss': 0.0458, 'grad_norm': 2.1255369186401367, 'learning_rate': 1e-05, 'epoch': 3.42}
{'loss': 0.0563, 'grad_norm': 2.0731728076934814, 'learning_rate': 1e-05, 'epoch': 3.77}
{'loss': 0.0469, 'grad_norm': 2.2

  0%|          | 0/32 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.06385818123817444, 'eval_wer_ortho': 235.32751091703057, 'eval_wer': 254.7265625, 'eval_runtime': 435.6833, 'eval_samples_per_second': 1.15, 'eval_steps_per_second': 0.073, 'epoch': 6.85}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


{'train_runtime': 3265.8599, 'train_samples_per_second': 2.45, 'train_steps_per_second': 0.153, 'train_loss': 0.5471928314119577, 'epoch': 6.85}


TrainOutput(global_step=500, training_loss=0.5471928314119577, metrics={'train_runtime': 3265.8599, 'train_samples_per_second': 2.45, 'train_steps_per_second': 0.153, 'train_loss': 0.5471928314119577, 'epoch': 6.85})

In [24]:
trainer.save_model("./saved_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfi

events.out.tfevents.1710392712.DN51uspn.SUNet.89907.0:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

In [25]:
kwargs = {
    # "dataset_tags": "mozilla-foundation/common_voice_13_0",
    "dataset": "Latex",  # a 'pretty' name for the training dataset
    "language": "en",
    "model_name": "Whisper Small Latex - Lichu",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "text-to-speech",
}

trainer.push_to_hub(**kwargs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/lichuacu/whisper-small-latex/commit/c6b3495d039e4c666362ab680b3d788f23225630', commit_message='End of training', commit_description='', oid='c6b3495d039e4c666362ab680b3d788f23225630', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
from transformers import pipeline

pipe = pipeline(model="lichuacu/whisper-small-latex") 

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.86k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [76]:
idx = 34
text = pipe(inputs=dataset["test"][idx]["audio"])
print(f"True: {dataset['test'][idx]['sentence']}")
print(f"Predicted: {text['text']}")


True:  \alpha_1, b_2,..., \alpha_n \in \mathbb{{set}}
Predicted: {\alpha1, b2, __________________________________________ ______{\alphan ______________________ ______{\alphan __________________ ______{\alphan ______________ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______ ______
